<a href="https://colab.research.google.com/github/praveensurya2808/Atm_machine/blob/main/Numpy_leanings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --quiet

import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: praveensurya28
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 954MB/s]

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader,Dataset
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)



cuda


In [ ]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()
data_df.dropna(inplace=True)
data_df.drop(['id'],axis=1,inplace=True)
data_df.head()
data_df.shape

(18185, 11)

In [ ]:
data_df['Class'].value_counts()

data_df['Class'].unique()


array([1, 0])

In [ ]:
original_df = data_df.copy()
for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()
data_df.head()

In [ ]:
X = np.array(data_df.iloc[:,:-1]) #data_df.iloc[:,:-1]
Y = np.array(data_df.iloc[:,-1])

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
X_test,X_val,Y_test,Y_val = train_test_split(X_test,Y_test,test_size = 0.5)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
class dataset(Dataset):
  def __init__(self,X,Y):
    self.X= torch.tensor(X,dtype = torch.float32).to(device)
    self.Y= torch.tensor(Y,dtype = torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index],self.Y[index]



In [ ]:
training_data = dataset(X_train,Y_train)
val_data = dataset(X_val,Y_val)
testing_data = dataset(X_test,Y_test)


In [ ]:
training_dataloader = DataLoader(training_data,batch_size= 7, shuffle= True)
testing_dataloader = DataLoader(testing_data,batch_size= 7, shuffle= True)
validation_dataloader = DataLoader(val_data,batch_size= 7, shuffle= True)

In [ ]:
for x, y in training_dataloader:
  print(x)
  print("========")
  print(y)

In [ ]:
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel,self).__init__()

    self.input_layer = nn.Linear(X.shape[1],HIDDEN_NEURONS)
    self.linear = nn.Linear(HIDDEN_NEURONS,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x= self.input_layer(x)
    x= self.linear(x)
    x= self.sigmoid(x)
    return x

model = MyModel().to(device)

In [ ]:
summary(model,(X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
